In [0]:
storage_account = "stocsocailogs"
storage_key = "PASTE YOUR KEY HERE"

spark.conf.set(f"fs.azure.account.key.{storage_account}.dfs.core.windows.net", storage_key)

dbutils.fs.ls(f"abfss://raw@{storage_account}.dfs.core.windows.net/")



[FileInfo(path='abfss://raw@stocsocailogs.dfs.core.windows.net/security_events.csv', name='security_events.csv', size=347, modificationTime=1767730970000)]

In [0]:
storage_account = "stocsocailogs"
storage_key = "PASTE YOUR KEY HERE"

container = "raw"
file_name = "security_events.csv"

# Set storage key
spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    storage_key
)

# Build ADLS path
path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{file_name}"

# Read CSV
df = spark.read.option("header", True).csv(path)

display(df)



timestamp,event_type,severity,source_ip,username
2025-01-01 09:12:33,LoginFailure,High,203.0.113.45,jdoe
2025-01-01 09:15:02,LoginSuccess,Low,10.0.0.5,asmith
2025-01-01 09:18:45,MalwareDetected,Critical,198.51.100.23,system
2025-01-01 09:22:10,PortScan,Medium,192.0.2.10,unknown
2025-01-01 09:30:55,LoginFailure,High,203.0.113.45,jdoe
null,null,null,null,null


In [0]:
from pyspark.sql.functions import col, to_timestamp

df_silver = (
    df
    .withColumn("event_time", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss"))
    .drop("timestamp")
    .filter(col("event_time").isNotNull())
    .filter(col("event_type").isNotNull())
    .filter(col("severity").isNotNull())
    .filter(col("source_ip").isNotNull())
    .filter(col("username").isNotNull())
)

display(df_silver)



event_type,severity,source_ip,username,event_time
LoginFailure,High,203.0.113.45,jdoe,2025-01-01T09:12:33Z
LoginSuccess,Low,10.0.0.5,asmith,2025-01-01T09:15:02Z
MalwareDetected,Critical,198.51.100.23,system,2025-01-01T09:18:45Z
PortScan,Medium,192.0.2.10,unknown,2025-01-01T09:22:10Z
LoginFailure,High,203.0.113.45,jdoe,2025-01-01T09:30:55Z


In [0]:
from pyspark.sql.functions import when

df_silver = (
    df_silver
    .withColumn(
        "severity_score",
        when(col("severity") == "Low", 1)
        .when(col("severity") == "Medium", 2)
        .when(col("severity") == "High", 3)
        .when(col("severity") == "Critical", 4)
        .otherwise(0)
    )
)

display(df_silver)



event_type,severity,source_ip,username,event_time,severity_score
LoginFailure,High,203.0.113.45,jdoe,2025-01-01T09:12:33Z,3
LoginSuccess,Low,10.0.0.5,asmith,2025-01-01T09:15:02Z,1
MalwareDetected,Critical,198.51.100.23,system,2025-01-01T09:18:45Z,4
PortScan,Medium,192.0.2.10,unknown,2025-01-01T09:22:10Z,2
LoginFailure,High,203.0.113.45,jdoe,2025-01-01T09:30:55Z,3


In [0]:
silver_path = f"abfss://curated@{storage_account}.dfs.core.windows.net/silver_security_events"

(df_silver
 .write
 .mode("overwrite")
 .format("delta")
 .save(silver_path)
)

print("✅ Silver Delta saved at:", silver_path)


✅ Silver Delta saved at: abfss://curated@stocsocailogs.dfs.core.windows.net/silver_security_events


In [0]:
df_silver_check = spark.read.format("delta").load(silver_path)
display(df_silver_check)



event_type,severity,source_ip,username,event_time,severity_score
LoginFailure,High,203.0.113.45,jdoe,2025-01-01T09:12:33Z,3
LoginSuccess,Low,10.0.0.5,asmith,2025-01-01T09:15:02Z,1
MalwareDetected,Critical,198.51.100.23,system,2025-01-01T09:18:45Z,4
PortScan,Medium,192.0.2.10,unknown,2025-01-01T09:22:10Z,2
LoginFailure,High,203.0.113.45,jdoe,2025-01-01T09:30:55Z,3
